In [83]:
import music21
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"
music21.converter.parse(little_star).show('mid')

In [84]:
# 데이터  sampling
# ABC 표기 데이터는 2채널(계이름과 박자), 분류문제
# x : [ [1,4],[1,4],[1,4] ... ]
# y : [4 4]- 원핫 000000000100000

In [85]:
import numpy as np

In [86]:
temp = ['c','d','e','f','g','a','b']
note2num = dict(zip(temp,np.arange(1,8)))
num2note =  { value:key for key,value in note2num.items()  }
note2num, num2note

({'c': 1, 'd': 2, 'e': 3, 'f': 4, 'g': 5, 'a': 6, 'b': 7},
 {1: 'c', 2: 'd', 3: 'e', 4: 'f', 5: 'g', 6: 'a', 7: 'b'})

In [87]:
# 시계열 데이터 만들기
def abc2timeseries(s):
    return [ [note2num[i[0]], int(i[1]) ] for i in s.split()[2:] ]
temp = abc2timeseries(little_star)
temp[:5]

[[1, 4], [1, 4], [5, 4], [5, 4], [6, 4]]

In [88]:
def timeseries2abc(t):
    return "tinynotation: 4/4 "+" ".join([num2note[i]+str(j) for i,j in t])
timeseries2abc(temp)    

'tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2'

In [89]:
# 원핫 코드로 변환하는 표
#[  [1,2],[2,2],..[7,2], ]
onehot = [[i,2] for i in range(1,8)] + [[i,4] for i in range(1,8)] + [[i,8] for i in range(1,8)]

In [90]:
seq = abc2timeseries(little_star)
seq[:5]

[[1, 4], [1, 4], [5, 4], [5, 4], [6, 4]]

In [91]:
# 시계열 데이터 만들기
w = 8  # 윈도우 크기
h = 1  # 수평 계수
def seq2dadtaset(seq,w,h):
    x,y = [],[]
    for i in range(len(seq)-(w+h)+1):        
        x.append(seq[i:(i+w)])
        y.append(seq[i+w+h-1])
    return np.array(x), np.array(y)   

In [92]:
X,Y = seq2dadtaset(seq,w,h)
print(X.shape, Y.shape)
print(X[0], Y[0])

(34, 8, 2) (34, 2)
[[1 4]
 [1 4]
 [5 4]
 [5 4]
 [6 4]
 [6 4]
 [5 2]
 [4 4]] [4 4]


In [93]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
import tensorflow as tf

In [94]:
# 훈련데이터
split = int(len(X)*1.0)  # 100% 훈련집합으로 사용
x_train = X[:split]; y_train = Y[:split]

In [95]:
t = []
def to_onehot(y):
    for i in range(len(y)):
        a = np.zeros(len(onehot))
        a[onehot.index(list(y[i])  )] = 1.0    # 예측이 목적이 아니라 생성이 목적이기 때문에 전체를 훈련집합으로
        t.append(a)
    return np.array(t)

In [96]:
y_train = to_onehot(y_train)

In [97]:
y_train

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0.,

In [98]:
x_train.shape,y_train.shape,x_train[0].shape

((34, 8, 2), (34, 21), (8, 2))

In [125]:
# 모델 학습
model = Sequential([
    LSTM(units=128,activation='relu',input_shape = x_train[0].shape),  # (8,2)
    Dense(y_train.shape[1],activation='softmax'),  # 출력은 y의 원핫코드의 길이 21    
])

In [126]:
model.compile(loss = tf.keras.losses.categorical_crossentropy,optimizer='Adam',metrics=['acc'])

In [127]:
model.fit(x_train,y_train,epochs=10,batch_size=1)

Epoch 1/10
34/34 [==============================] - 1s 5ms/step - loss: 2.9850 - acc: 0.0882
Epoch 2/10
34/34 [==============================] - 0s 5ms/step - loss: 2.2537 - acc: 0.1765
Epoch 3/10
34/34 [==============================] - 0s 5ms/step - loss: 2.0441 - acc: 0.1765
Epoch 4/10
34/34 [==============================] - 0s 4ms/step - loss: 1.9864 - acc: 0.2353
Epoch 5/10
34/34 [==============================] - 0s 4ms/step - loss: 1.9063 - acc: 0.3824
Epoch 6/10
34/34 [==============================] - 0s 5ms/step - loss: 1.7304 - acc: 0.4412
Epoch 7/10
34/34 [==============================] - 0s 4ms/step - loss: 1.7893 - acc: 0.2353
Epoch 8/10
34/34 [==============================] - 0s 4ms/step - loss: 1.4925 - acc: 0.5000
Epoch 9/10
34/34 [==============================] - 0s 4ms/step - loss: 1.5351 - acc: 0.3824
Epoch 10/10
34/34 [==============================] - 0s 5ms/step - loss: 1.4702 - acc: 0.4412


In [135]:
new_song = []
for i in range(32):
    p = model.predict(np.expand_dims(x_train[i],axis=0),verbose=0)
    new_song.append( onehot[np.argmax(p)])
new_song[:10]    

[[3, 4],
 [3, 4],
 [3, 4],
 [3, 4],
 [2, 4],
 [2, 4],
 [5, 4],
 [5, 4],
 [4, 4],
 [4, 4]]

In [136]:
timeseries2abc(new_song)

'tinynotation: 4/4 e4 e4 e4 e4 d4 d4 g4 g4 f4 f4 f4 f4 e4 e4 g4 g4 e4 e4 e4 e4 d4 g4 g4 g4 g4 f4 f4 f4 e4 e4 e4 e4'

In [137]:
music21.converter.parse(timeseries2abc(new_song)).show('mid')